In [15]:
import pymysql
import requests
import pandas as pd
import numpy as np

### Genera un df con el archivo `csv` "CHARTEVENTS"

In [17]:
df = pd.read_csv('C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\CHARTEVENTS-7.csv', encoding='UTF-8', delimiter=',', decimal='.')
df.head(3)

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,272587133,41976,130681,265505.0,227375,30/10/2200 08:11,30/10/2200 08:11,17574,No,NaN,NaN,0,0,NaN,NaN
1,272587134,41976,130681,265505.0,228096,30/10/2200 08:11,30/10/2200 08:11,17574,-1 Awakens to voice (eye opening/contact) > 10...,-1.0,NaN,0,0,NaN,NaN
2,272587135,41976,130681,265505.0,226381,30/10/2200 08:27,30/10/2200 08:27,20889,Married,2.0,NaN,0,0,NaN,NaN


In [18]:
df = df.replace(np.nan, 0)

### Se conecta con la BD MySQL AWS

In [ ]:
con = pymysql.connect(
      host='database-proyecto-final.cpjrl88bhdul.us-east-1.rds.amazonaws.com',
      database='databaseuci',
      user='admin',
      password='datascience'
)

### Inserta los registros del df "CHARTEVENTS" en la BD

In [19]:
sql_insert = ''' INSERT 
                   INTO  CHARTEVENTS
                         (row_id, subject_id, hadm_id, icustay_id, itemid, charttime, storetime, cgid, value, valuenum,
                          valueuom, warning, error, resultstatus, stopped)  
                 VALUES  (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) '''
num_reng = 0
cursor = con.cursor()
for i in range(0, len(df)):
    row = int(df.loc[i, 'row_id'])
    subj = int(df.loc[i, 'subject_id'])
    hadm = int(df.loc[i, 'hadm_id'])
    icus = int(df.loc[i, 'icustay_id'])
    itid = int(df.loc[i, 'itemid'])
    chtm = df.loc[i, 'charttime']
    sttm = df.loc[i, 'storetime']
    cgid = int(df.loc[i, 'cgid'])
    valu = df.loc[i, 'value']
    valn = int(df.loc[i, 'valuenum'])
    valm = df.loc[i, 'valueuom']
    warn = int(df.loc[i, 'warning'])
    erro = int(df.loc[i, 'error'])
    rest = df.loc[i, 'resultstatus']
    stop = df.loc[i, 'stopped']
    reg_s = (row, subj, hadm, icus, itid, chtm, sttm, cgid, valu, valn, valm, warn, erro, rest, stop)
    cursor.execute(sql_insert, reg_s)
    num_reng += 1
    if num_reng == 100:
        con.commit()
        num_reng = 0

KeyboardInterrupt: 

### Confirma la inserción de los registros a la BD

In [20]:
con.commit()

InterfaceError: (0, '')

### Cierra el cursor y la conexion a la BD

In [14]:
cursor.close()
con.close()